In [5]:
## Samsung VD sample App source model
## Author : Siddeshwar MK ( s.mk@samsung.com )
## Date  :  2023/05/21

import os
import numpy as np
import matplotlib as mpl
import pandas as pd
import seaborn as sns
import sklearn
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import pytz
from matplotlib import pyplot as plt
import glob

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import concatenate
from keras.layers import Input
from sklearn.metrics import classification_report
from matplotlib import pyplot    

csv_file = 'appsrc_test_sample_data.csv'
cwd = os.getcwd()
test_df = pd.read_csv(csv_file,index_col=None, header=0)
test_df.info()

def make_cnn_model(metrics,output_bias=None,lr_rate = 0.001):
    
    print('CNN model build with leanring_rate :',lr_rate)
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    prev_app_open_embeddings_stacked = keras.Input(shape=(10,5), dtype=tf.float32, name="prev_app_open_embeddings_stacked")
    time_sin = keras.Input(shape=(1,),dtype=tf.float32, name="time_sine_value")
    time_cos = keras.Input(shape=(1,),dtype=tf.float32, name="time_cos_value")
    day_sin = keras.Input(shape=(1,),dtype=tf.float32, name="day_sine_value")
    day_cos = keras.Input(shape=(1,),dtype=tf.float32, name="day_cos_value")
    x = prev_app_open_embeddings_stacked
    for i in range(num_conv_layers):
        x = layers.Conv1D(filters=conv_filters, kernel_size=3, activation="relu")(x)
    x = layers.Flatten()(x)
    x = layers.Concatenate(axis=1)([x,time_sin,time_cos,day_sin,day_cos])
    x = layers.Dense(100, activation="relu",name="Dense1")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(50, activation="relu",name="Dense2")(x)
    inputs = [prev_app_open_embeddings_stacked, time_sin, time_cos, day_sin, day_cos]
    outputs = layers.Dense(num_prev_apps+1, activation='softmax',bias_initializer=output_bias)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        optimizer=keras.optimizers.Adam(learning_rate=lr_rate),
        metrics=metrics)
    return model


def prepapre_cnn_model_test_data(test_df) :

     # Data prepare 
    #print(test_df.info())
    X_test = test_df.iloc[:,:50].to_numpy().reshape(len(test_df),10,5)
  
    X_test_t_sin = test_df['time_sin'].to_numpy() 
    X_test_t_cos = test_df['time_cos'].to_numpy()
  
    X_test_d_sin = test_df['day_sin'].to_numpy() 
    X_test_d_cos = test_df['day_cos'].to_numpy()
  
    X_test=np.array(X_test)
    X_test = np.squeeze(X_test)
  
    test_features = [X_test,X_test_t_sin,X_test_t_cos,X_test_d_sin,X_test_d_cos]
    #print('Feature dim :', len(test_features))
    return test_features

def prepare_test_labels(test_df):
    from keras.utils import np_utils
    test_labels = np_utils.to_categorical(test_df['labels'])
    return test_labels


def plot_test_class_confusion_matrix(model,test_features,test_labels):
    test_predictions_baseline = model.predict(test_features, batch_size=16) 
    cnf_matrix_report = classification_report(np.argmax(test_labels, axis=1), np.argmax(test_predictions_baseline, axis=1),digits=3)
    print(cnf_matrix_report) 
    cnf_matrix = confusion_matrix(np.argmax(test_labels, axis=1), np.argmax(test_predictions_baseline, axis=1))
    print(cnf_matrix)
    
    
def model_evaluate(model,test_features,test_labels):
    baseline_results = model.evaluate(test_features, test_labels,
                                  batch_size=16, verbose=0)
    for name, value in zip(model.metrics_names, baseline_results):
        print(name, ': ', value)
    print()
    
    
def test_appsrc_model_device(train_data,model,lr_rate=.001,batch_size=64,number_epochs=50):
    
    test_features = prepapre_cnn_model_test_data(test_df)
    test_labels = prepare_test_labels(test_df)
    
    model_evaluate(model,test_features, test_labels)
    plot_test_class_confusion_matrix(model,test_features, test_labels)
    
cnn_model_name = 'cnn_caf_appsrc_model_0615_4week_5_x'
cnn_model_file = cnn_model_name+'.h5'
print('Model file: ',cnn_model_file)
app_model = tf.keras.models.load_model(cnn_model_file)  
app_model.summary()
print('Model layer weights: ')
app_weights = app_model.weights
print(app_weights)
test_appsrc_model_device(test_df,app_model)  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 55 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   prev_1hr_count1         44 non-null     float64
 1   prev_7hr_count2         44 non-null     float64
 2   prev_24hr_count3        44 non-null     float64
 3   prev_72hr_count4        44 non-null     float64
 4   prev_168hr_count5       44 non-null     float64
 5   recurrence_count6       44 non-null     float64
 6   24hr_duration_ratio7    44 non-null     float64
 7   72hr_duration_ratio8    44 non-null     float64
 8   168hr_duration_ratio9   44 non-null     float64
 9   time_distance_ratio10   44 non-null     float64
 10  prev_1hr_count11        44 non-null     float64
 11  prev_7hr_count12        44 non-null     float64
 12  prev_24hr_count13       44 non-null     float64
 13  prev_72hr_count14       44 non-null     float64
 14  prev_168hr_count15      44 non-null     floa

loss :  1.3819788694381714
precision@1 :  0.6136363744735718
precision@2 :  0.40909090638160706
recall@1 :  0.6136363744735718
recall@2 :  0.8181818127632141
accuracy@1 :  0.6136363744735718
accuracy@2 :  0.8181818127632141
top5accuracy@5 :  0.9545454382896423

3/3 [==============================] - 0s 1ms/step


/home/seungbaek/miniconda3/envs/torch1.13/lib/python3.10/site-packages/keras/backend.py:5561: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


              precision    recall  f1-score   support

           0      0.731     0.731     0.731        26
           1      0.500     0.500     0.500         6
           2      0.375     0.500     0.429         6
           3      1.000     0.500     0.667         2
           4      0.333     0.500     0.400         2
           5      0.000     0.000     0.000         2

    accuracy                          0.614        44
   macro avg      0.490     0.455     0.454        44
weighted avg      0.612     0.614     0.607        44

[[19  1  4  0  2  0]
 [ 2  3  1  0  0  0]
 [ 1  2  3  0  0  0]
 [ 1  0  0  1  0  0]
 [ 1  0  0  0  1  0]
 [ 2  0  0  0  0  0]]


/home/seungbaek/miniconda3/envs/torch1.13/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/seungbaek/miniconda3/envs/torch1.13/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/seungbaek/miniconda3/envs/torch1.13/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [11]:
import numpy as np

csv_file = 'appsrc_test_sample_data.csv'
cwd = os.getcwd()
test_df = pd.read_csv(csv_file,index_col=None, header=0)
arr = test_df.to_numpy()

print(arr.shape)
test_df.head()

(44, 55)


,prev_1hr_count1,prev_7hr_count2,prev_24hr_count3,prev_72hr_count4,prev_168hr_count5,recurrence_count6,24hr_duration_ratio7,72hr_duration_ratio8,168hr_duration_ratio9,time_distance_ratio10,...,recurrence_count46,24hr_duration_ratio47,72hr_duration_ratio48,168hr_duration_ratio49,time_distance_ratio50,time_sin,time_cos,labels,day_sin,day_cos
0,1.0,2.0,2.0,12.0,28.0,3.0,0.034708,0.066438,0.056763,0.991228,...,1.0,0.0,0.020315,0.024874,0.0,0.650288,-0.759688,0,-0.279415,0.960170
1,1.0,1.0,1.0,11.0,27.0,1.0,0.043661,0.070408,0.058036,0.973451,...,1.0,0.0,0.020315,0.024874,0.0,0.990607,0.136737,0,-0.279415,0.960170
2,1.0,0.0,0.0,10.0,26.0,1.0,0.000000,0.073589,0.058807,0.981818,...,1.0,0.0,0.020315,0.024874,0.0,-0.999990,0.004426,0,-0.279415,0.960170
3,1.0,0.0,0.0,1.0,2.0,0.0,0.000000,0.035282,0.050831,0.907407,...,1.0,0.0,0.020315,0.024874,0.0,0.412118,-0.911130,1,-0.279415,0.960170
4,1.0,3.0,6.0,13.0,29.0,0.0,0.072155,0.065385,0.057900,0.989796,...,1.0,0.0,0.020315,0.024874,0.0,-0.846220,-0.532833,3,-0.958924,0.283662


In [21]:
import time

##
# @brief save data into file with filename
# @param[in] data The data to be saved
def save(filename, layers_of_model):
    if os.path.isfile(filename):
        os.remove(filename)

    for weights in layers_of_model:
        
        with open(filename, 'ab') as outfile:
            print(weights.shape, "weights are saved.")
            for item in weights:
              np.array(item, dtype=np.float32).tofile(outfile)
        
layers_of_model = app_model.get_weights()
save("./weights.bin", layers_of_model)

(3, 5, 10) weights are saved.
(10,) weights are saved.
(3, 10, 10) weights are saved.
(10,) weights are saved.
(3, 10, 10) weights are saved.
(10,) weights are saved.
(44, 100) weights are saved.
(100,) weights are saved.
(100, 50) weights are saved.
(50,) weights are saved.
(50, 6) weights are saved.
(6,) weights are saved.


(3, 5, 10)